# Crypto Arbitrage

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

from datetime import timedelta

import ipywidgets as widgets
from IPython.display import display

import collection
import vizualization as vz
# %matplotlib inline

## Collect the Data

### Bitstamp

In [2]:
bitstamp = collection.Data_Container()
bitstamp.df = bitstamp.read_csv(Path("Resources/bitstamp.csv"))
bitstamp.clean_data()

### Coinbase

In [3]:
coinbase = collection.Data_Container()
coinbase.df =  coinbase.read_csv(Path("Resources/coinbase.csv"))
coinbase.clean_data()

## Analyze the Data

### Bitstamp

In [4]:
# Slice the DataFrames for the timestamp and 'Close' price 
bitstamp.slice_data()
# bitstamp_sliced = bitstamp.sliced

# Generate the summary statistics for the DataFrame
bitstamp_summary = bitstamp.df.describe(include='all')


### Coinbase

In [5]:
# Slice the DataFrames for the timestamp and 'Close' price 
coinbase.slice_data()
# coinbase_sliced = coinbase.sliced

# Generate the summary statistics for the DataFrame
coinbase_summary = coinbase.df.describe(include='all')

In [6]:
main_df = pd.DataFrame(data=[bitstamp.sliced, coinbase.sliced], index=['Bitstamp', 'Coinbase']).T
display(main_df)

,Bitstamp,Coinbase
Timestamp,,
2018-01-01 00:00:00,13646.48,13608.49
2018-01-01 00:01:00,13658.75,13601.66
2018-01-01 00:02:00,13610.22,13580.00
2018-01-01 00:03:00,13639.09,13550.34
2018-01-01 00:04:00,13620.00,13583.44
...,...,...
2018-03-31 23:55:00,6922.56,6930.00
2018-03-31 23:56:00,6920.32,6930.01
2018-03-31 23:57:00,6934.72,6933.91


In [7]:
main_df['Spread'] = (main_df['Bitstamp'] - main_df['Coinbase']).abs()

In [8]:
def calc_arbitrage_percent(df):
    if df['Bitstamp'] < df['Coinbase']:
        return 100 * df['Spread'] / df['Bitstamp']
    elif df['Bitstamp'] > df['Coinbase']:
        return 100 * df['Spread'] / df['Coinbase']
    else:
        return 0.00

In [9]:
main_df['Return %'] = main_df.apply(calc_arbitrage_percent, axis=1)

In [10]:
# Review the spread return DataFrame
display(main_df)

,Bitstamp,Coinbase,Spread,Return %
Timestamp,,,,
2018-01-01 00:00:00,13646.48,13608.49,37.99,0.279164
2018-01-01 00:01:00,13658.75,13601.66,57.09,0.419728
2018-01-01 00:02:00,13610.22,13580.00,30.22,0.222533
2018-01-01 00:03:00,13639.09,13550.34,88.75,0.654965
2018-01-01 00:04:00,13620.00,13583.44,36.56,0.269151
...,...,...,...,...
2018-03-31 23:55:00,6922.56,6930.00,7.44,0.107475
2018-03-31 23:56:00,6920.32,6930.01,9.69,0.140022
2018-03-31 23:57:00,6934.72,6933.91,0.81,0.011682


In [11]:
def gen_sliced_df(date_picked):
    slice_start_date = bitstamp.sliced.index[date_picked] - timedelta(days=.5)
    slice_end_date = slice_start_date + timedelta(days=1)

    sliced_df = main_df.loc[slice_start_date:slice_end_date]

    return sliced_df

In [12]:
slice_date_1 = gen_sliced_df(10000)
display(slice_date_1)

,Bitstamp,Coinbase,Spread,Return %
Timestamp,,,,
2018-01-07 10:59:00,16443.00,16502.01,59.01,0.358876
2018-01-07 11:00:00,16445.00,16502.00,57.00,0.346610
2018-01-07 11:01:00,16468.27,16502.01,33.74,0.204879
2018-01-07 11:02:00,16383.57,16490.00,106.43,0.649614
2018-01-07 11:03:00,16372.68,16450.00,77.32,0.472250
...,...,...,...,...
2018-01-08 10:55:00,14549.99,14613.85,63.86,0.438901
2018-01-08 10:56:00,14617.82,14646.85,29.03,0.198593
2018-01-08 10:57:00,14550.00,14600.01,50.01,0.343711


In [13]:
scale = widgets.FloatRangeSlider(
    value = [.75*min(main_df['Bitstamp']), 1.25*max(main_df['Bitstamp'])],
    min=.75*min(main_df['Bitstamp']), 
    max=1.25*max(main_df['Bitstamp']),
    description='Scale',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=False,
    readout_format='d',
    # layout=widgets.Layout(height='350px'),
    layout=widgets.Layout(height='100%'),
)

In [14]:
bitstamp_checkbox = widgets.Checkbox(
    value=True,
    description='Bitstamp',
    disabled=False
)

coinbase_checkbox = widgets.Checkbox(
    value=True,
    description='Coinbase',
    disabled=False
)


In [15]:
date_slicer = widgets.IntRangeSlider(
    value=[0, len(main_df.index)-1],
    min=0,
    max=len(main_df.index)-1,
    step=1,
    description='Date Range',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


In [16]:
# date_picker = widgets.IntSlider(
#     value=int(round(len(main_df.index)/2, 0)),
#     min=date_slicer.value[0],
#     max=date_slicer.value[1],
#     description='Date Picker',
#     readout=False,
#     layout=widgets.Layout(width='900px')
#     )

date_picker_1 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    layout=widgets.Layout(width='900px')
    )

date_picker_2 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    layout=widgets.Layout(width='900px')
    )

date_picker_3 = widgets.IntSlider(
    value=int(round(len(main_df.index)/2, 0)),
    min=date_slicer.value[0],
    max=date_slicer.value[1],
    description='Date Picker',
    readout=False,
    layout=widgets.Layout(width='900px')
    )


In [17]:
# date_picked = widgets.Text(
#     f"{main_df.index[date_picker.value]}",
#     )

date_picked_1 = widgets.Text(
    f"{main_df.index[date_picker_1.value]}",
    )

date_picked_2 = widgets.Text(
    f"{main_df.index[date_picker_2.value]}",
    )

date_picked_3 = widgets.Text(
    f"{main_df.index[date_picker_3.value]}",
    )


In [18]:
def main_plotter(
    # date_picker_slider, 
    range_slider, 
    date_slicer, 
    bitstamp_checkbox, 
    coinbase_checkbox, 
    # scale
    ):
    """
    A helper function to make a graph.
    """
    # date_picker.min = date_slicer[0]
    # date_picker.max = date_slicer[1]
    # date_picked.value = f"{main_df.index[date_picker_slider-1]}"

    vz.main_plotter_2(
        main_df, 
        # date_picker_slider, 
        range_slider, 
        date_slicer, 
        bitstamp_checkbox, 
        coinbase_checkbox, 
        # scale
        )


main_plot = widgets.interactive_output(
    main_plotter, 
    {
        # 'date_picker_slider': date_picker,
        'range_slider': scale,
        'date_slicer': date_slicer,
        'bitstamp_checkbox': bitstamp_checkbox,
        'coinbase_checkbox': coinbase_checkbox,
        # 'scale': scale,
        }
    )


In [19]:
def day_plotter(date_picker):
    vz.day_plotter_2(main_df, date_picker)
    
# day_out = widgets.interactive_output(
#     day_plotter,
#     {
#         'date_picker': date_picker,
#     }
# )

day_out_1 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_1,
    }
)

day_out_2 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_2,
    }
)

day_out_3 = widgets.interactive_output(
    day_plotter,
    {
        'date_picker': date_picker_3,
    }
)


In [20]:
def arbitrage_summary(target_date_index):
    sliced_df = gen_sliced_df(target_date_index)
    vz.arbitrage_summary(sliced_df['Spread'])

arbitrage_out_1 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_1
    }
)

arbitrage_out_2 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_2
    }
)

arbitrage_out_3 = widgets.interactive_output(
    arbitrage_summary,
    {
        'target_date_index': date_picker_3
    }
)


In [21]:
tab_1_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_1, 
                date_picked_1
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_1, 
                day_out_1
            ]
        ), 
    ]
) 

tab_2_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_2, 
                date_picked_2
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_2, 
                day_out_2
            ]
        ), 
    ]
) 

tab_3_container = widgets.VBox(
    [
        widgets.HBox(
            [
                date_picker_3, 
                date_picked_3
            ]
        ), 
        widgets.HBox(
            [
                arbitrage_out_3, 
                day_out_3
            ]
        ), 
    ]
) 

tab_titles = ['First Tab', 'Second Tab', 'Third Tab']

children = [tab_1_container, tab_2_container, tab_3_container]
tab = widgets.Tab()
for i, t in enumerate(tab_titles):
    tab.set_title(i, t)
tab.children = children


In [22]:
widgets.VBox(
    [
        widgets.HBox([date_slicer, widgets.VBox([bitstamp_checkbox, coinbase_checkbox])]), 
        widgets.HBox([scale, main_plot]), 
        tab,
        ],
    )

In [23]:
# slice_date_1 = gen_sliced_df(date_picker_1.value)
# slice_date_2 = gen_sliced_df(date_picker_2.value)
# slice_date_3 = gen_sliced_df(date_picker_3.value)
